In [1]:
import os, sys
import subprocess

subprocess.check_call([sys.executable, "-m", "pip", "install", "import_ipynb"])
import import_ipynb

from google.colab import drive
drive.mount('/content/gdrive')
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/ModularNeuralNetwork')

import numpy as np
import matplotlib.pyplot as plt

from Function_Wrapper import function_wrapper
from Activation_Functions import ReLU, ReLU_prime, tanh, tanh_prime, sigmoid, sigmoid_prime, identity, identity_prime, softmax
from Cost_Functions import mse, mse_prime, cross_entropy, cross_entropy_prime
from Initialization_Strategies import rand, normal, Xavier, He, none

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
importing Jupyter notebook from Function_Wrapper.ipynb
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
importing Jupyter notebook from Activation_Functions.ipynb
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
importing Jupyter notebook from Cost_Functions.ipynb
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
importing Jupyter notebook from Initialization_Strategies.ipynb
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
def adam_variations(layer, index, adam_variation = "Adam"):
  ###
  # Function to implement Adam, Adamax, and Nadam solvers
  #
  # Parameters:
  # layer- Current layer to update weights and biases
  # index- Index of the current batch
  # adam_variation- Choice between Adam, Adamax, and Nadam
  ###


  dW = layer.weights_gradient # Variables to store the weights and biases gradients found in backpropagation
  dB = layer.biases_gradient

  layer.adam_parameters.m_weights = layer.adam_parameters.beta_0 * layer.adam_parameters.m_weights + (1 - layer.adam_parameters.beta_0) * dW # Update m values for weights
  
  layer.adam_parameters.m_biases = layer.adam_parameters.m_biases * layer.adam_parameters.beta_0 # Update m values for biases
  layer.adam_parameters.m_biases = layer.adam_parameters.m_biases + ((1 - layer.adam_parameters.beta_0) * dB)

  m_hat_weights = layer.adam_parameters.m_weights / (1 - np.power(layer.adam_parameters.beta_0, index)) # find m_hat for weights and biases
  m_hat_biases = layer.adam_parameters.m_biases / (1 - np.power(layer.adam_parameters.beta_0, index))

  if(adam_variation == "Adam" or adam_variation == "Nadam"): # Adam and Nadam share more operations, so they are both included here
    layer.adam_parameters.v_weights = layer.adam_parameters.beta_1 * layer.adam_parameters.v_weights + (1 - layer.adam_parameters.beta_1) * np.power(dW, 2) # Calculate v values
    layer.adam_parameters.v_biases = layer.adam_parameters.beta_1 * layer.adam_parameters.v_biases + (1 - layer.adam_parameters.beta_1) * dB
    
    if(adam_variation == "Nadam"): # Additional update to m_hat if Nadam is being used
      m_hat_weights += (1 - layer.adam_parameters.beta_0) * dW / (1 - np.power(layer.adam_parameters.beta_0, index))
      m_hat_biases += (1 - layer.adam_parameters.beta_0) * dB / (1 - np.power(layer.adam_parameters.beta_0, index)) 

    v_hat_weights = layer.adam_parameters.v_weights / (1 - np.power(layer.adam_parameters.beta_1, index)) # Calculate v_hat for weights and biases
    v_hat_biases = layer.adam_parameters.v_biases / (1 - np.power(layer.adam_parameters.beta_1, index))
    
    layer.weights -= layer.eta * (m_hat_weights / (np.sqrt(v_hat_weights) + layer.adam_parameters.epsilon)) # Update weights and biases
    layer.biases -= layer.eta * (m_hat_biases / (np.sqrt(v_hat_biases) + layer.adam_parameters.epsilon))

  elif(adam_variation == "Adamax"):
    layer.adam_parameters.v_weights = np.maximum(layer.adam_parameters.beta_1 * layer.adam_parameters.v_weights, np.abs(dW))
    layer.adam_parameters.v_biases = np.maximum(layer.adam_parameters.beta_1 * layer.adam_parameters.v_biases, np.abs(dB)) 
    
    layer.weights = layer.weights - (layer.eta * (m_hat_weights / (layer.adam_parameters.v_weights + layer.adam_parameters.epsilon)))
    layer.biases = layer.biases - (layer.eta * (m_hat_biases / (layer.adam_parameters.v_biases + layer.adam_parameters.epsilon)))

  else:
    raise Exception("Given value is not Adam or a variation thereof")
  
# Presets for each variation, made with function_wrapper function in Function_Wrapper.ipynb  
standard_Adam = function_wrapper(adam_variations, adam_variation = "Adam")
Nadam = function_wrapper(adam_variations, adam_variation = "Nadam")
Adamax = function_wrapper(adam_variations, adam_variation = "Adamax")

In [3]:
def solver(input_layer, X, y, epochs, batch_proportion = .5, solver_type = "SGD", adam_variation = None):
  ###
  # Function to implement various optimizers. Includes gradient descent, stochastic gradient descent, and variations of Adam
  #
  # Parameters:
  # input_layer- Input layer of the network
  # X- Feature set from the current batch
  # y- Classifications for the current batch
  # batch_proportion- Proportion of total training data made up by the current batch
  # solver_type- Standard, SGD, or Adam. If Adam is chosen, the computation will be handeled by the adam_variations function regardless of variation choice
  # adam_variation- Adam, Adamax, or Nadam
  ###

  implemented_solvers = ["SGD", "standard", "Adam", "Nadam", "Adamax"] # List of implemented optimizers
  output_layer = input_layer.get_output_layer() # find the output layer of the network given the input layer

  if(solver_type == "SGD"): # SGD calculation

    num_samples = len(y)
    batch_size = int(batch_proportion * num_samples) # Calculate size of batch

    for index in range(epochs):
      indices = np.random.choice(num_samples, size = batch_size) # Find random subset of training data of size batch_size
      subset_X = X[indices]; subset_y = y[indices]

      output_layer.batch_backward_propagate(subset_X, subset_y, index) # Backpropagate for current batch
      output_layer.update_parameters() # Update weights and biases 
      
  elif(solver_type == "Adam"): # Set up Adam calculation, delegate computation of final values to adam_variations

    num_samples = len(y)
    batch_size = int(batch_proportion * num_samples) # Calculate size of batch 

    for index in range(epochs): 
      indices = np.random.choice(num_samples, size = batch_size) # Find random subset of training data of size batch_size
      subset_X = X[indices]; subset_y = y[indices]

      output_layer.batch_backward_propagate(subset_X, subset_y, index) # Backpropagate

      layer = output_layer
      while(layer.prev_layer != None): # Perform the chosen Adam variation on each layer
        adam_variation(layer, index + 1)
        layer = layer.prev_layer

  elif(solver_type == "standard"): # Standard gradient descent

    for index in range(epochs):
      for X_i, y_i in zip(X, y):
        output_layer.batch_backward_propagate(X_i, y_i, index)
        output_layer.update_parameters()
    
  else:
    raise NotImplementedError("Solver not implemented, currently implemented functions are:" + str(implemented_solvers))

# Presets for solvers
standard_solver = function_wrapper(solver, solver_type = "standard")
SGD_solver = function_wrapper(solver, solver_type = "SGD")
Adam_solver = function_wrapper(solver, solver_type = "Adam", adam_variation = standard_Adam)
Nadam_solver = function_wrapper(solver, solver_type = "Adam", adam_variation = Nadam)
Adamax_solver = function_wrapper(solver, solver_type = "Adam", adam_variation = Adamax)

In [4]:
class Adam_Parameters: # Class to store parameters of Adam calculations
  def __init__(self, layer, beta_0 = .9, beta_1 = .999, epsilon = .0001):
    self.layer = layer
    self.weights_shape = np.shape(self.layer.weights)
    self.biases_shape = np.shape(self.layer.biases)

    self.m_weights = np.zeros(self.weights_shape)
    self.v_weights = self.m_weights
    self.m_biases = np.zeros(self.biases_shape)
    self.v_biases = self.m_biases

    self.beta_0 = beta_0
    self.beta_1 = beta_1
    self.epsilon = epsilon

In [8]:
class Layer: # Layer class stores a single layer of the network and performs forward and backward propagation
  def __init__(self, activation = None, activation_prime = None, prev_layer = None, num_inputs = 0, num_nodes = None, eta = .01, batch_normalize = True):
    ###
    # Initialize layer class
    #
    # Parameters:
    # activation- Chosen activation function. One of the presets created in Activation_Functions.ipynb should be passed here 
    # activation_prime- Derivative of chosen activation function. One of the presets created in Activation_Functions.ipynb should be passed here 
    # prev_layer- Previous layer in network
    # num_inputs- Number of nodes in the previous layer, of number of features per sample if this is the input layer
    # num_nodes- Number of nodes in this layer
    # eta- Learning rate
    # batch_normalize- Perform batch normalization if true
    ### 

    self.activation = activation
    self.activation_prime = activation_prime

    self.initialization_strategy = None
    self.set_initialization_strategy()

    self.weights = self.initialization_strategy(num_nodes, num_inputs)
    self.biases = np.zeros(num_nodes)

    self.num_nodes = num_nodes
    self.num_inputs = num_inputs

    self.inputs = []
    self.z = []
    self.a = []
    self.predictions = []

    self.error_out = []
    self.weights_gradient = []
    self.biases_gradient = []

    self.next_layer = None
    self.prev_layer = prev_layer

    self.eta = eta
    self.alpha = 1
    self.adam_parameters = Adam_Parameters(self)

  def set_initialization_strategy(self): 
    ###
    # Choose weight initialization strategy based on this layer's activation function
    ###

    if(self.activation == None):
      self.initialization_strategy = none
    elif(self.activation == ReLU):
      self.initialization_strategy = He
    else:
      self.initialization_strategy = Xavier
    
  def get_output_layer(self):
    ###
    # Find output layer given the input layer
    ###

    if(self.next_layer == None):
      return self
    return self.next_layer.get_output_layer()

  def get_input_layer(self):
    ###
    # Find input layer given the output layer
    ###

    if(self.prev_layer == None):
      return self
    return self.prev_layer.get_input_layer()

  def batch_forward_propagate(self, X, store_data = True):
    ###
    # Function to perform forward propagation for a given batch
    #
    # Parameters:
    # X- Samples in current batch
    # store_data- Whether or not to store values for backpropagation. Will be false when testing, true when training
    ###

    if(self.prev_layer != None): # Ensure that function was called on the correct layer
      raise Exception("batch_forward_propagate() can only be be called on input layer")

    def forward_propagate(layer, X_i, store_data = True):
      ###
      # Function to forward propagate a single sample
      #
      # Parameters:
      # layer- The current layer
      # X_i- single sample from batch
      # store_data- described above
      ###

      inputs = np.asarray(X_i)
        
      if(layer.prev_layer == None): # Check if the current layer is the input layer
        z = inputs
        a = z
          
      else: # Otherwise, calculate z and a values
        z = np.dot(layer.weights, inputs) + layer.biases
        a = layer.activation(z)

      if(store_data): # Store values if needed for back propagation
        layer.inputs.append(inputs)
        layer.z.append(z)
        layer.a.append(a)

      if(layer.next_layer != None): # Check if current layer is output layer, otherwise recursively call forward_propagate on the next layer
        forward_propagate(layer.next_layer, a, store_data = store_data)

      layer.predictions.append(a)

    current_layer = self
    while(current_layer != None): # Reset layer values
      current_layer.inputs = []
      current_layer.z = []
      current_layer.a = []
      current_layer.error_out = []
      current_layer = current_layer.next_layer

    for sample in np.atleast_2d(X): # Call forward_propagate
      forward_propagate(self, sample, store_data)

    output_layer = self.get_output_layer()
    if(not store_data): # If store_data is false, final predictions are stored
      output = output_layer.predictions[-np.shape(X)[0]:] # Remove last output of network so it won't be included in backpropagation
      return output

    return np.asarray(output_layer.a) # Return a values of output layer

  def batch_backward_propagate(self, X, y, epoch_count):
    ###
    # Function to perform backward propagation for a given batch
    #
    # Parameters:
    # X- Samples in current batch
    # y- Labels in current batch
    # epoch_count- current epoch
    ###

    if(self.next_layer != None): # Ensure that function was called on the correct layer
      raise Exception("batch_backward_propagate() can only be be called on output layer")

    num_samples = np.shape(X)[0]
    indices = np.arange(num_samples)

    input_layer = self.get_input_layer()
    a = input_layer.batch_forward_propagate(X, store_data = True).flatten() # Forward propagte the batch
    y = np.asarray(y)
    a = np.reshape(a, np.shape(y))
    deltas = a - y # Calculate output layer errors
    
    if(epoch_count%100 == 0):
      print("epoch", epoch_count, "err:", mse(y, a), "derErr:", sum(a-y)/len(y))

    def backward_propagate(layer, error, index):
      ###
      # Function to find errors for a single sample
      #
      # Parameters:
      # layer- The current layer
      # error- Error from the following layer
      # index- Index of the individual sample
      ###

      if(layer.next_layer == None): # Check if output layer
        new_error = np.asarray(error).flatten()
        layer.error_out.append([new_error])

      elif(layer.prev_layer != None):
        coefficients = np.asarray(layer.next_layer.weights).T # Transposition of the next layer's weights
        errors = []
        new_error = np.zeros(np.shape(layer.next_layer.weights)[1])

        for row in coefficients:
          error_to_add = np.dot(error, row) * layer.activation_prime(layer.z[index]) # Dot product of next layer error and weights times derivative of current layer z values
          errors.append(error_to_add)
          new_error = new_error + error_to_add

        layer.error_out.append(errors)

      if(layer.prev_layer != None): # Recursively call backward_propagate on the previous layer
        backward_propagate(layer.prev_layer, new_error, index)

    def compute_gradients(layer, divisor):
      ###
      # Function to compute gradients of a layer based on its error
      #
      # Parameters:
      # layer- current layer
      # divisor- numbber of samples in batch for averaging
      ###

      if(layer.prev_layer != None): # Ensure current layer is not input
        weights_gradient = np.zeros(np.shape(layer.weights))
        final_deltas = []

        for delta, current_input, current_z in zip(layer.error_out, layer.inputs, layer.z):
          delta_tmp = []
          for lis in delta: # This loop is a temporary fix for some numpy issues, will be removed later
            delta_tmp.append(np.asarray(lis.flatten()))
            final_deltas.append(np.asarray(lis.flatten()))
          delta = np.asarray(delta_tmp)
          
          for err in delta: # Calculate gradient for each error
            grad = np.dot(current_input[np.newaxis].T, np.asarray(err)[np.newaxis])
            weights_gradient = weights_gradient + grad.T
        
        weights_gradient = weights_gradient
        #weights_gradient /= divisor

        biases_gradient = np.mean(np.asarray(final_deltas), 0)

        layer.weights_gradient = weights_gradient # Set gradients for instance of layer class
        layer.biases_gradient = biases_gradient

        compute_gradients(layer.prev_layer, divisor) # Recursive call on previous layer

    
    for delta, sample_index in zip(np.atleast_2d(deltas), indices): # Backpropagate for each sample
      backward_propagate(self, delta, sample_index)

    compute_gradients(self, num_samples)
    

  
  def update_parameters(self): # Update weights and biases for the layer
    if(self.prev_layer != None):
      self.prev_layer.update_parameters()
      self.weights = self.weights - (self.weights_gradient * self.eta)
      self.biases = self.biases - (self.biases_gradient * self.eta) 


In [9]:
class Network:
  def __init__(self, solver = Adam_solver, activation = ReLU, activation_prime = ReLU_prime, eta = .01, batch_proportion = .25):
    ###
    # Initialize neural net
    #
    # Parameters:
    # solver- chosen solver
    # activation- Chosen activation function. One of the presets created in Activation_Functions.ipynb should be passed here 
    # activation_prime- Derivative of chosen activation function. One of the presets created in Activation_Functions.ipynb should be passed here 
    # eta- Learning rate
    # batch_proportion- Value to calculate size of each batch
    ###

    self.solver = solver
    self.activation = activation
    self.activation_prime = activation_prime

    self.input_layer = []
    self.output_layer = []
    self.layer_dimensions = []   
    
    self.batch_proportion = batch_proportion
    self.eta = eta
    
  def create_network(self, layer_dimensions):
    ###
    # Function to create the layers of the network
    #
    # Parameters:
    # layer_dimensions- List storing the dimensions of the network. The first element of the list
    # will store the number of features per sample of the training data, the final element stores the
    # number of possible class labels to assign. Elements of the list between the first and last will
    # be the desired number of nodes in each layer of the network.
    ###

    self.layer_dimensions = layer_dimensions
    length = len(self.layer_dimensions)
    new_layer = None
    prev_layer = None

    self.input_layer = Layer(eta = None, num_inputs = layer_dimensions[0])
    prev_layer = self.input_layer

    for index in range(length-1):

        new_layer = Layer(prev_layer = prev_layer,
                          num_inputs = self.layer_dimensions[index],
                          num_nodes = self.layer_dimensions[index+1],
                          activation = self.activation,
                          activation_prime = self.activation_prime)
        
        new_layer.prev_layer.next_layer = new_layer
        prev_layer = new_layer

    self.output_layer = prev_layer
    self.output_layer.activation = softmax # Output layer uses softmax activation
    self.output_layer.activation_prime = identity_prime # No need to program softmax_prime, it cancels out with the correct loss function

  def predict(self, X):
    ###
    # Function to predict class labels
    #
    # Parameters:
    # X- List of samples to predict
    ###

    predictions = self.input_layer.batch_forward_propagate(X, store_data = False)
    predictions_tmp = []
    for lis in predictions:
      predictions_tmp.append(np.argmax(np.asarray(lis.flatten())))
    predictions = np.asarray(predictions_tmp)
    return predictions

  def fit(self, X, y, epochs):
    ###
    # Simply calls the chosen solver
    #
    # Parameters:
    # X- Training data features
    # y- Training data labels
    # epochs- Number of epochs to run
    ###
    
    self.solver(self.input_layer, X, y, epochs, self.batch_proportion)

In [10]:
# This block of code simply tests the network on performing an exclusive-or operation for simple and quick testing

X = np.array([[0,0], [0,1], [1,0], [1,1]])
y = np.array([[1,0], [0,1], [0,1], [1,0]])
y_argmax = np.array([np.argmax(y_i) for y_i in y])

xorNetwork = Network(eta = .1, solver = Adamax_solver, activation = ReLU, activation_prime = ReLU_prime, batch_proportion = 1)
xorNetwork.create_network([2, 10, 2])

xorNetwork.fit(X, y, 2000)

preds = xorNetwork.predict(X)
print("Predictions:\n",preds,"\nactual:\n", y_argmax) # Print the class labels as predicted by the network and the actual class labels

epoch 0 err: 0.3223023617095754 derErr: [ 0.30836182 -0.30836182]
epoch 100 err: 0.24541987414321287 derErr: [ 0.05209539 -0.05209539]
epoch 200 err: 0.23568312463691046 derErr: [-0.25388157  0.25388157]
epoch 300 err: 0.16834035328408298 derErr: [ 0.28837905 -0.28837905]
epoch 400 err: 0.0902718083442398 derErr: [ 0.1480076 -0.1480076]
epoch 500 err: 0.1737977596142922 derErr: [ 0.05276686 -0.05276686]
epoch 600 err: 0.15339107744575128 derErr: [-0.30049791  0.30049791]
epoch 700 err: 0.2071445900476923 derErr: [-0.4551314  0.4551314]
epoch 800 err: 0.06140048876755964 derErr: [ 0.05195484 -0.05195484]
epoch 900 err: 0.0345885690670213 derErr: [ 0.05576657 -0.05576657]
epoch 1000 err: 0.027737763654773186 derErr: [ 0.1022093 -0.1022093]
epoch 1100 err: 0.008954868346945123 derErr: [-0.01454362  0.01454362]
epoch 1200 err: 0.0075428266192705835 derErr: [ 0.03813434 -0.03813434]
epoch 1300 err: 0.003164060660605318 derErr: [-0.00332878  0.00332878]
epoch 1400 err: 0.0033454641647577225 